In [51]:
from functools import partial, reduce
from os import rename, makedirs
import urllib.request
import pandas as pd
import os.path
import shutil
import json
import csv
import re
import datetime

In [74]:
def gsheet_download_csv(id: str, target: str, url_template="https://docs.google.com/spreadsheets/d/{id}/export?format=csv"):
    url = url_template.format(id=id)
    return urllib.request.urlretrieve(url, target)

gsheet_download_csv('1kK1Yu6gz5kEWe_i0vamiGttkXUH5H90e', 'nomenclador.csv')
paises = pd.read_csv('./nomenclador.csv')
pais = {k:v for k,v in paises[['iso3', 'iso3_desc_fundar']].iloc}
codes_set = set(pais.keys())

lista_cambios = []
diccionario_cambios = {}

In [85]:
def match_relocate(src, target, pattern, file_list=None):
    files_to_move = file_list or os.listdir(src)

    pattern = re.compile(pattern)

    if not os.path.exists(target):
        makedirs(target)

    for filename in files_to_move:
        if pattern.match(filename):
            src_path = os.path.join(src, filename)
            dest_path = os.path.join(target, filename)

            shutil.move(src_path, dest_path)

    print(f"Files matching the pattern '{pattern}' moved to '{target}'.")

def compose2(f, g):
    def composed(*a, **kw):
        return f(g(*a, **kw))

    return composed

def compose(*fs):
    return reduce(compose2, fs)

def _chain(*fs):
    return compose(*reversed(fs))

def chain(*fs):
    lista_cambios.clear()
    return _chain(*fs)

def exportar_definitivo(archivo: str, folder:str, df: pd.DataFrame, nuevo_nombre=None):
    if not nuevo_nombre:
        nuevo_nombre = f"{folder}/definitivos/{archivo}_old.csv"

    original_file = f'{folder}/definitivos/{archivo}.csv'

    rename(original_file, nuevo_nombre)

    df.to_csv(original_file, 
              encoding='utf-8', 
              sep=',', 
              quoting=csv.QUOTE_ALL, 
              quotechar='"', 
              lineterminator='\n', 
              decimal='.', 
              index=False)
    
    return original_file, nuevo_nombre

def drop_col(df: pd.DataFrame, col, axis=1, lista_cambios = lista_cambios):
    lista_cambios.append(f"Se borró la columna {col}")
    return df.drop(col, axis=axis)

def drop_colx(col, axis=1):
    return lambda df: drop_col(df, col, axis=axis)

def normalize_countries(df: pd.DataFrame, code_col:str, new_col:str, desc_col:str, map: dict, codes_set=codes_set, lista_cambios = lista_cambios):
    today = datetime.datetime.now().strftime("%Y-%m-%d")
    diff_set = set(df[code_col].dropna()) - codes_set
    if len(diff_set)>0: 
        diff_str = ", ".join(list(diff_set))
        string = f"ERROR: Los códigos {diff_str} no se encontraron en el nomenclador vigente al {today} en la columna {code_col}"
        lista_cambios.append(string)
    else:
        if new_col:
            df[new_col] = df[code_col].apply(map.__getitem__)
            df = df.drop(desc_col, axis=1)
            string = f"Usando los valores de la columna {code_col}, 
            se normalizaron los nombres de las regiones/paises con el nomenclador vigente al {today}, 
            creando la columna {new_col} y borrando la columna {desc_col}"
        else: 
            df[desc_col] = df[code_col].apply(map.__getitem__)
            string = f"Usando los valores de la columna {code_col}, 
            se normalizaron los nombres de las regiones/paises con el nomenclador vigente al {today}, 
            cambiando los valores de la columna {desc_col}"
        lista_cambios.append(string)
    return df

def normalize_countriesx(code_col, new_col, desc_col, map):
    return lambda df: normalize_countries(df, code_col=code_col, new_col=new_col, desc_col=desc_col, map=map)

def wide_to_long(df: pd.DataFrame, primary_keys, value_name='valor', var_name='indicador', lista_cambios=lista_cambios):
    cols = ", ".join(df.columns)
    new_cols = ", ".join(primary_keys + [value_name, var_name])
    string = f"Long estricto: [{cols}] -> [{new_cols}]"
    lista_cambios.append(string)
    return df.melt(id_vars=primary_keys, value_name=value_name, var_name=var_name)

def wide_to_longx(primary_keys, value_name='valor', var_name='indicador'):
    return lambda df: wide_to_long(df, primary_keys, value_name=value_name, var_name=var_name)

def rename_cols(df: pd.DataFrame, map, lista_cambios=lista_cambios):
    map_string = ", ".join(f"{old_col}:{new_col}" for old_col, new_col in map.items())
    string = f"Se renombraron las columnas: {map_string}"
    lista_cambios.append(string)
    df = df.rename(columns=map)
    return df

def rename_colsx(map):
    return lambda df: rename_cols(df, map)

def replace_value(df:pd.DataFrame, col:str, curr_value:str, new_value:str, lista_cambios=lista_cambios):
    string = f"Se imputó el valor {new_value} en la columna {col} cuando {col}=={curr_value}"
    lista_cambios.append(string)
    df = df.replace({col: curr_value}, new_value)
    return df

def replace_valuex(col, curr_value, new_value):
    return lambda df: replace_value(df=df, col=col, curr_value=curr_value, new_value=new_value)

def sort_vals_asc(df:pd.DataFrame, prim_keys:list, lista_cambios=lista_cambios):
    lista_cambios.append(f"Ordené las filas por las columnas: {', '.join(prim_keys)}")
    return df.sort_values(by=prim_keys).reset_index(drop=True)

def sort_vals_ascx(prim_keys):
    return lambda df: sort_vals_asc(df=df, prim_keys=prim_keys)

COMEXT_g1.csv

In [76]:
SUBTOP = 'COMEXT'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 1

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio


,anio,cantidades_exportacion_ferreres,cantidades_exportacion_indec
0,1810,0.019382,NaN
1,1811,0.024362,NaN
2,1812,0.010767,NaN
3,1813,0.013730,NaN
4,1814,0.018324,NaN


In [77]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)


pipeline = chain(
    to_long(primary_keys),
    sort_vals_ascx(primary_keys)
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'region_code': 'cod_area'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)


diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,indicador,valor
0,1810,cantidades_exportacion_ferreres,0.019382
1,1810,cantidades_exportacion_indec,NaN
2,1811,cantidades_exportacion_ferreres,0.024362
3,1811,cantidades_exportacion_indec,NaN
4,1812,cantidades_exportacion_indec,NaN


In [78]:
diccionario_cambios

{'COMEXT_g1': ['Long estricto: [anio, cantidades_exportacion_ferreres, cantidades_exportacion_indec] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio']}

In [79]:
exportar(_df)

('../output/COMEXT1/definitivos/COMEXT_g1.csv',
 '../output/COMEXT1/definitivos/COMEXT_g1_old.csv')

COMEXT_g2.csv

In [80]:
SUBTOP = 'COMEXT'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 2

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

time iso3


,time,iso3,countryname,exportsconstant_goods_v2,exportsconstant_servi_v2
0,2000,ABW,Aruba,NaN,NaN
1,2007,ABW,Aruba,NaN,NaN
2,1964,ABW,Aruba,NaN,NaN
3,2021,ABW,Aruba,NaN,NaN
4,2016,ABW,Aruba,NaN,NaN


In [81]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3', new_col="desc_fundar", desc_col="countryname"),
    # drop(col="countryname"),
    to_long(primary_keys + ['desc_fundar']),
    sort(primary_keys),
    renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,cod_fundar,desc_fundar,indicador,valor
0,1960,ABW,Aruba,exportsconstant_goods_v2,NaN
1,1960,ABW,Aruba,exportsconstant_servi_v2,NaN
2,1960,AFE,África Oriental y del Sur,exportsconstant_goods_v2,NaN
3,1960,AFE,África Oriental y del Sur,exportsconstant_servi_v2,NaN
4,1960,AFG,Afganistán,exportsconstant_goods_v2,NaN


In [82]:
diccionario_cambios

{'COMEXT_g1': ['Long estricto: [anio, cantidades_exportacion_ferreres, cantidades_exportacion_indec] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'COMEXT_g2': ['Se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-08',
  'Long estricto: [time, iso3, exportsconstant_goods_v2, exportsconstant_servi_v2, desc_fundar] -> [time, iso3, desc_fundar, valor, indicador]',
  'Ordené las filas por las columnas: time, iso3',
  'Se renombraron las columnas: iso3:cod_fundar, time:anio']}

In [83]:
exportar(_df)

('../output/COMEXT1/definitivos/COMEXT_g2.csv',
 '../output/COMEXT1/definitivos/COMEXT_g2_old.csv')

COMEXT_g3.csv

In [86]:
SUBTOP = 'COMEXT'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 3

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

time iso3


,time,iso3,countryname,servicesexportsbop_pc_v2
0,2000,ABW,Aruba,87.073685
1,2007,ABW,Aruba,22.031572
2,1964,ABW,Aruba,NaN
3,2021,ABW,Aruba,94.411430
4,2016,ABW,Aruba,87.884972


In [87]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3', new_col="desc_fundar", desc_col="countryname"),
    # drop(col="countryname"),
    to_long(primary_keys + ['desc_fundar']),
    sort(primary_keys),
    renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,cod_fundar,desc_fundar,indicador,valor
0,1960,ABW,Aruba,servicesexportsbop_pc_v2,NaN
1,1960,AFE,África Oriental y del Sur,servicesexportsbop_pc_v2,NaN
2,1960,AFG,Afganistán,servicesexportsbop_pc_v2,NaN
3,1960,AFW,África Occidental y Central,servicesexportsbop_pc_v2,NaN
4,1960,AGO,Angola,servicesexportsbop_pc_v2,NaN


In [88]:
diccionario_cambios

{'COMEXT_g1': ['Long estricto: [anio, cantidades_exportacion_ferreres, cantidades_exportacion_indec] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'COMEXT_g2': ['Se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-08',
  'Long estricto: [time, iso3, exportsconstant_goods_v2, exportsconstant_servi_v2, desc_fundar] -> [time, iso3, desc_fundar, valor, indicador]',
  'Ordené las filas por las columnas: time, iso3',
  'Se renombraron las columnas: iso3:cod_fundar, time:anio'],
 'COMEXT_g3': ['Se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-08',
  'Long estricto: [time, iso3, servicesexportsbop_pc_v2, desc_fundar] -> [time, iso3, desc_fundar, valor, indicador]',
  'Ordené las filas por las columnas: time, iso3',
  'Se renombraron las columnas: iso3:cod_fundar, time:anio']}

In [90]:
exportar(_df)

('../output/COMEXT1/definitivos/COMEXT_g3.csv',
 '../output/COMEXT1/definitivos/COMEXT_g3_old.csv')

In [ ]:
match_relocate(f'{folder}/definitivos', f'{folder}/old', '.*old.*')

In [5]:
# import json

# with open('../output/COMEXT1/definitivos/paso_a_definitivos_20240311.json', 'w') as fp:
#     json.dump(a, fp, indent=4, ensure_ascii=False)
